# **Read the CSV File with PySpark**

In [ ]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,918 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,161 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,518 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
!pip install pyspark
!pip install py4j

In [ ]:
!unzip /content/drive/MyDrive/MachineLearning/Loan_default.csv.zip

Archive:  /content/drive/MyDrive/MachineLearning/Loan_default.csv.zip
  inflating: Loan_default.csv        


In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session (if not already done)
spark = SparkSession.builder \
    .appName("LoanDefaultAnalysis") \
    .getOrCreate()

# Read the CSV file
df = spark.read.csv(r"/content/Loan_default.csv", header=True, inferSchema=True)

# Show the schema and first few rows
df.printSchema()
df.show(5)

root
 |-- LoanID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- MonthsEmployed: integer (nullable = true)
 |-- NumCreditLines: integer (nullable = true)
 |-- InterestRate: double (nullable = true)
 |-- LoanTerm: integer (nullable = true)
 |-- DTIRatio: double (nullable = true)
 |-- Education: string (nullable = true)
 |-- EmploymentType: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- HasMortgage: string (nullable = true)
 |-- HasDependents: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- HasCoSigner: string (nullable = true)
 |-- Default: integer (nullable = true)

+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+
|    LoanID|Age|Income

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data Cleaning**

In [ ]:
df = df.drop("LoanID")
df.show(5)

+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+
|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|  Education|EmploymentType|MaritalStatus|HasMortgage|HasDependents|LoanPurpose|HasCoSigner|Default|
+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+
| 56| 85994|     50587|        520|            80|             4|       15.23|      36|    0.44| Bachelor's|     Full-time|     Divorced|        Yes|          Yes|      Other|        Yes|      0|
| 69| 50432|    124440|        458|            15|             1|        4.81|      60|    0.68|   Master's|     Full-time|      Married|         No|           No|      Other|        Yes|      0|
| 46| 84208|    1291

In [ ]:
from pyspark.sql.functions import round,col

# Create Debt-to-Income feature
df = df.withColumn("DebtAmount", round(col("Income") * col("DTIRatio"), 2))

### Checking Missing Value

In [ ]:
from pyspark.sql.functions import col, sum as spark_sum

# Count null values in each column
null_counts = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+---+------+----------+-----------+--------------+--------------+------------+--------+--------+---------+--------------+-------------+-----------+-------------+-----------+-----------+-------+----------+
|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|Education|EmploymentType|MaritalStatus|HasMortgage|HasDependents|LoanPurpose|HasCoSigner|Default|DebtAmount|
+---+------+----------+-----------+--------------+--------------+------------+--------+--------+---------+--------------+-------------+-----------+-------------+-----------+-----------+-------+----------+
|  0|     0|         0|          0|             0|             0|           0|       0|       0|        0|             0|            0|          0|            0|          0|          0|      0|         0|
+---+------+----------+-----------+--------------+--------------+------------+--------+--------+---------+--------------+-------------+-----------+-------------+-----------+-------

### Checking for Duplicate Rows

In [ ]:
total_rows = df.count()
distinct_rows = df.distinct().count()

print(f"Total rows: {total_rows}")
print(f"Distinct rows: {distinct_rows}")
print(f"Duplicate rows: {total_rows - distinct_rows}")

Total rows: 255347
Distinct rows: 255347
Duplicate rows: 0


### converting binary categorical columns (with "Yes"/"No" values) into numerical columns (1/0 integers).

In [ ]:
from pyspark.sql.functions import when,col

binary_cols = ["HasMortgage", "HasDependents", "HasCoSigner"]
for col in binary_cols:
    df = df.withColumn(col,
                      when(df[col] == "Yes", 1)
                      .otherwise(0)
                      .cast("integer"))

In [ ]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- MonthsEmployed: integer (nullable = true)
 |-- NumCreditLines: integer (nullable = true)
 |-- InterestRate: double (nullable = true)
 |-- LoanTerm: integer (nullable = true)
 |-- DTIRatio: double (nullable = true)
 |-- Education: string (nullable = true)
 |-- EmploymentType: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- HasMortgage: integer (nullable = false)
 |-- HasDependents: integer (nullable = false)
 |-- LoanPurpose: string (nullable = true)
 |-- HasCoSigner: integer (nullable = false)
 |-- Default: integer (nullable = true)
 |-- DebtAmount: double (nullable = true)



In [ ]:
df.select(["LoanPurpose"]).distinct().show()

+-----------+
|LoanPurpose|
+-----------+
|  Education|
|       Home|
|      Other|
|       Auto|
|   Business|
+-----------+



In [ ]:
from pyspark.sql.functions import col, when

loan_purposes = [row.LoanPurpose for row in df.select("LoanPurpose").distinct().collect()]

for purpose in loan_purposes:
    df = df.withColumn(f"LoanPurpose_{purpose}", when(col("LoanPurpose") == purpose, 1).otherwise(0))

df.show(5)

+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+----------+---------------------+----------------+-----------------+----------------+--------------------+
|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|  Education|EmploymentType|MaritalStatus|HasMortgage|HasDependents|LoanPurpose|HasCoSigner|Default|DebtAmount|LoanPurpose_Education|LoanPurpose_Home|LoanPurpose_Other|LoanPurpose_Auto|LoanPurpose_Business|
+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+----------+---------------------+----------------+-----------------+----------------+--------------------+
| 56| 85994|     50587|        520|            80|             4|       15.23|      36|    0.4

In [ ]:
df.select(["MaritalStatus"]).distinct().show()

+-------------+
|MaritalStatus|
+-------------+
|      Married|
|     Divorced|
|       Single|
+-------------+



In [ ]:
df = df.withColumn("MaritalStatus_Married",when(col("MaritalStatus") == "Married", 1).otherwise(0))
df = df.withColumn("MaritalStatus_Single",when(col("MaritalStatus") == "Single", 1).otherwise(0))
df = df.withColumn("MaritalStatus_Divorced",when(col("MaritalStatus") == "Divorced", 1).otherwise(0))

In [ ]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- MonthsEmployed: integer (nullable = true)
 |-- NumCreditLines: integer (nullable = true)
 |-- InterestRate: double (nullable = true)
 |-- LoanTerm: integer (nullable = true)
 |-- DTIRatio: double (nullable = true)
 |-- Education: string (nullable = true)
 |-- EmploymentType: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- HasMortgage: integer (nullable = false)
 |-- HasDependents: integer (nullable = false)
 |-- LoanPurpose: string (nullable = true)
 |-- HasCoSigner: integer (nullable = false)
 |-- Default: integer (nullable = true)
 |-- DebtAmount: double (nullable = true)
 |-- LoanPurpose_Education: integer (nullable = false)
 |-- LoanPurpose_Home: integer (nullable = false)
 |-- LoanPurpose_Other: integer (nullable = false)
 |-- LoanPurpose_Auto: integer (nullable = false)
 |-- LoanPurpo

In [ ]:
df.select(["EmploymentType"]).distinct().show()

+--------------+
|EmploymentType|
+--------------+
|     Part-time|
| Self-employed|
|     Full-time|
|    Unemployed|
+--------------+



In [ ]:
from pyspark.sql.functions import when, col

df = df.withColumn("EmploymentType",
                   when(col("EmploymentType") == "Unemployed", 0)
                   .when(col("EmploymentType") == "Part-time", 1)
                   .when(col("EmploymentType") == "Full-time", 2)
                   .when(col("EmploymentType") == "Self-employed", 3)
                   .otherwise(None)
                   .cast("integer"))

In [ ]:
df.show()

+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+----------+---------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+----------------------+
|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|  Education|EmploymentType|MaritalStatus|HasMortgage|HasDependents|LoanPurpose|HasCoSigner|Default|DebtAmount|LoanPurpose_Education|LoanPurpose_Home|LoanPurpose_Other|LoanPurpose_Auto|LoanPurpose_Business|MaritalStatus_Married|MaritalStatus_Single|MaritalStatus_Divorced|
+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+----------+---------------------+----------------+-----------------+-

In [ ]:
df = df.drop("MaritalStatus","LoanPurpose")

In [ ]:
df.show()

+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-----------+-------------+-----------+-------+----------+---------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+----------------------+
|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|  Education|EmploymentType|HasMortgage|HasDependents|HasCoSigner|Default|DebtAmount|LoanPurpose_Education|LoanPurpose_Home|LoanPurpose_Other|LoanPurpose_Auto|LoanPurpose_Business|MaritalStatus_Married|MaritalStatus_Single|MaritalStatus_Divorced|
+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-----------+-------------+-----------+-------+----------+---------------------+----------------+-----------------+----------------+--------------------+---------------------+-------------------

In [ ]:
df.select(["Education"]).distinct().show()

+-----------+
|  Education|
+-----------+
|High School|
|        PhD|
| Bachelor's|
|   Master's|
+-----------+



In [ ]:
df = df.withColumn("Education_HighSchool",when(col("Education") == "High School", 1).otherwise(0))
df = df.withColumn("Education_Bachelor",when(col("Education") == "Bachelor's", 1).otherwise(0))
df = df.withColumn("Education_Master",when(col("Education") == "Master's", 1).otherwise(0))
df = df.withColumn("Education_PhD",when(col("Education") == "PhD", 1).otherwise(0))

In [ ]:
df.show(5)

+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-----------+-------------+-----------+-------+----------+---------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+----------------------+--------------------+------------------+----------------+-------------+
|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|  Education|EmploymentType|HasMortgage|HasDependents|HasCoSigner|Default|DebtAmount|LoanPurpose_Education|LoanPurpose_Home|LoanPurpose_Other|LoanPurpose_Auto|LoanPurpose_Business|MaritalStatus_Married|MaritalStatus_Single|MaritalStatus_Divorced|Education_HighSchool|Education_Bachelor|Education_Master|Education_PhD|
+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-----------+-------------+-----------+-------+-----

In [ ]:
df = df.drop("Education")

In [ ]:
df.show(5)

+---+------+----------+-----------+--------------+--------------+------------+--------+--------+--------------+-----------+-------------+-----------+-------+----------+---------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+----------------------+--------------------+------------------+----------------+-------------+
|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|EmploymentType|HasMortgage|HasDependents|HasCoSigner|Default|DebtAmount|LoanPurpose_Education|LoanPurpose_Home|LoanPurpose_Other|LoanPurpose_Auto|LoanPurpose_Business|MaritalStatus_Married|MaritalStatus_Single|MaritalStatus_Divorced|Education_HighSchool|Education_Bachelor|Education_Master|Education_PhD|
+---+------+----------+-----------+--------------+--------------+------------+--------+--------+--------------+-----------+-------------+-----------+-------+----------+---------------------+--------

In [ ]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- MonthsEmployed: integer (nullable = true)
 |-- NumCreditLines: integer (nullable = true)
 |-- InterestRate: double (nullable = true)
 |-- LoanTerm: integer (nullable = true)
 |-- DTIRatio: double (nullable = true)
 |-- EmploymentType: integer (nullable = true)
 |-- HasMortgage: integer (nullable = false)
 |-- HasDependents: integer (nullable = false)
 |-- HasCoSigner: integer (nullable = false)
 |-- Default: integer (nullable = true)
 |-- DebtAmount: double (nullable = true)
 |-- LoanPurpose_Education: integer (nullable = false)
 |-- LoanPurpose_Home: integer (nullable = false)
 |-- LoanPurpose_Other: integer (nullable = false)
 |-- LoanPurpose_Auto: integer (nullable = false)
 |-- LoanPurpose_Business: integer (nullable = false)
 |-- MaritalStatus_Married: integer (nullable = false)
 |-- MaritalStatus_Single: intege

In [ ]:
df.show()

+---+------+----------+-----------+--------------+--------------+------------+--------+--------+--------------+-----------+-------------+-----------+-------+----------+---------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+----------------------+--------------------+------------------+----------------+-------------+
|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|EmploymentType|HasMortgage|HasDependents|HasCoSigner|Default|DebtAmount|LoanPurpose_Education|LoanPurpose_Home|LoanPurpose_Other|LoanPurpose_Auto|LoanPurpose_Business|MaritalStatus_Married|MaritalStatus_Single|MaritalStatus_Divorced|Education_HighSchool|Education_Bachelor|Education_Master|Education_PhD|
+---+------+----------+-----------+--------------+--------------+------------+--------+--------+--------------+-----------+-------------+-----------+-------+----------+---------------------+--------

### We use coalesce() to get a single file

In [ ]:
df.coalesce(1).write.csv("spark_cleaned_data.csv", header=True)

### Things done in Spark :
1. Checked for missing value
2. Checked for duplicate rows
3. converted binary categorical columns (with "Yes"/"No" values) into numerical columns (1/0 integers).

In [ ]:
# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Export Pandas DataFrame to Excel
pandas_df.to_excel("spark_cleaned_data.xlsx", index=False)